# Create a Crypto Currency Volume Change  Alerting Tool to Detect Pumps on Bittrex

### Purpose of this tutorial
To create a customizable alerting tool to monitor volume changes accross all bittrex coins using LiveDataFrame
    
### What you will be creating:
An augmented trading tool that monitors volume changes accross an entire cryptocurrency exchange (bittrex) that:
  - Monitors all the coins in realtime
  - Has customizable alert criteria for volume change % and time (e.g "Alert when a coin's volume changes 10% in 10 minutes")
  - Plays a sound when customizable alert criteria is fulfilled
  - Displays a list of coins fulfilling the alert/volume criteria and a link to their chart.
  - Updates this list, checks volume changes of coins, and alerts and prints the list automatically every 7 seconds
    
### Prerequistes:
- Python3
- `livedataframe` package: `pip3 install livedataframe`
- Public and secret keys from [LiveDataFrame](https://app.livedataframe.com/users/new) (sign up is free)
- A can-do attitude

    
    
    


## Import Required Packages

- If you have not installed livedataframe please do so now: `pip3 install livedataframe`

In [ ]:
from livedataframe import LiveExchange, ExchangeInfo # The core packages used by this tutorial
import pandas as pd # import pandas for creating our new tool
from IPython.display import Audio, display, clear_output # for playing our alert sound and our monitoring tool
import time # for sleeping our tools loop

## Initialize the LiveExchange class for the Bittrex Exchange

The first thing we are going to do is get a list of active symbols on the bittrex exchange using the helper class `ExchangeInfo`

In [ ]:
bittrex_symbols = ExchangeInfo.list_symbols_for('bittrex')

Once we have our symbols, it is time to setup our live view of the Bittrex exchange using the class `LiveExchange`. Replace `<YOUR_PUBLIC_KEY_HERE>` and `<YOUR_SECRET_KEY_HERE>` with your actual public and secret livedataframe keys (as strings). 

These should have been emailed to you when you [signed up](https://app.livedataframe.com/users/new)

In [ ]:
live_bittrex = LiveExchange(
    public_key = "<YOUR_PUBLIC_KEY_HERE>",
    secret_key = "<YOUR_SECRET_KEY_HERE>",
    exchange = 'bittrex',
    symbols = bittrex_symbols, 
    lookback_period = '4h'
)

In the code snippet above we have initialized the `LiveExchange` class by telling it to 'watch' all symbols on the `Bittrex` exchange. The lookback_period of `4h` tells `LiveExchange` to pull the last 4 hours of historical data for each coin.

## Start the LiveExchange

After we have initialized the live exchange as `live_bittrex`, we must start it by calling `.start()` on the instance. 

Calling `.start()` loads the historical data, subscribes to websockets for each symbol, and then appends each new tick into its own dataframe giving you a live dataframe with a 4 hour window of data. Let's start the `LiveExchange` now

In [ ]:
live_bittrex.start()

When you first call `.start()` it can take anywhere from 10 seconds - 3 minutes (depending on your internet speed) to load the initial historical data. Once complete, you will have access to a dictionary of live DataFrames for every symbol traded on bittrex. 

To access one of these live dataframes use the `.symbols` attribute on the `live_bittrex` instance. For example, to access the live df for `BTC-ETH` we would specify `live_bittrex.symbols['BTC-ETH']`. This returns a dataframe that updates every 5 seconds. To see an update, wait 5 seconds and call it again.

Let's go ahead and access the livedataframe for the `BTC-ETH` symbol

## Access Your first Live DataFrame

In [ ]:
live_bittrex.symbols['BTC-ETH'].tail()

## Create a Function for Building the Volume Change Monitor

Let's actually start creating our augmented trading tool for monitoring volume changes. This function will return a DataFrame where the indexes are symbol names, and the volume change over the specified interval will be the column.

Let's go ahead and write a function do create this view. This function will take the `LiveExchange` instance (`live_bittrex` in this case) and the specified time interval as inputs. 

In [ ]:
def create_volume_monitor_dataframe(live_exchange, time_interval):
    
    output = {} # instantiate dictionary to hold our volume indicators
    
    count = 0
    
    # loop through every symbol and its corresponding LiveDataFrame in the live_exchange instance
    for symbol, live_dataframe in live_exchange.symbols.items():
        
        # truncate data to specified time interval     
        live_dataframe = live_dataframe.last(time_interval)  
        
        # total volume traded of symbol at the beginning of your specified time_interval 
        # E.g: if time_interval was '10m' this would be the volume traded 10 minutes ago
        beginning_volume = live_dataframe['Volume'].iloc[0] 
        
        # total current volume traded of the symbol
        current_volume = live_dataframe['Volume'].iloc[-1] 
        
        # calculate percent change of volume
        percent_change = ((current_volume - beginning_volume) / beginning_volume) * 100
        
        # store percent change
        output[symbol] = percent_change 
        
    # create volume monitor dataframe with indexes as symbols and column values as volume percent changes
    volume_monitor_df = pd.DataFrame.from_dict(output, orient = 'index') 
    
    # add column name to dataframe as the specifed interval
    volume_monitor_df.columns = [time_interval + " vol % chg"] 
    
    # return the volume monitor dataframe
    return volume_monitor_df 

## Run Function and Create Volume Monitor DataFrame
Now that we have created a function that builds a new dataframe that adds a "Vol % Change" column for our specified interval accross all bittrex assets, let's see the result. Let's take a look at the volume change for each asset over the last 1 hour and sort our dataframe so the coins with the highest volume changes are at the top

In [ ]:
# look at the volume change over 1 hour
volume_change_period = '1h'

# Create a volume monitor DataFrame
volume_monitor_df = create_volume_monitor_dataframe(
    live_bittrex, 
    volume_change_period
)

# Sort by largest change in volume
volume_monitor_df.sort_values(by = volume_change_period + " vol % chg", ascending = False)

As we can see, we have just created a dataframe that shows us the percent changes over the last 1 hour for each asset and ranks them by greatest volume change -- pretty cool huh? Now, let's go ahead and implement the volume threshold.

## Add a Volume Threshold Filter
Currently, our tool lists the change in volume for all of the coins on bittrex for a specified time period. There are a lot of coins on bittrex, 291 at the time of writing, so we better filter this list down. Let's make a function that will take our `volume_monitor_df` and a `volume_threshold` as parameters and then filter our dataframe down to symbols whose volume changes are greater than this threshold.

In [ ]:
def filter_volume_monitor_df_by_threshold(volume_monitor_df, volume_threshold):
    
    # select the rows from volume monitor dataframe where the change in volume is greater than threshold
    symbols_above_threshold = volume_monitor_df[volume_monitor_df > volume_threshold].dropna()  

    return symbols_above_threshold

## Use the Volume Threshold Filter
Let's go ahead and use this filter function now and filter all symbols that have had a volume change of at LEAST 20% in the last 1 hour.

In [ ]:
volume_monitor_df = create_volume_monitor_dataframe(live_bittrex, '1h')

filter_volume_monitor_df_by_threshold(volume_monitor_df, 20)

# Monitor Changes in Volume and get Realtime Alerts!
At this point we could keep running these two functions together over and over again and record the changes. This would allow us to 'monitor' the changes, but it wouldn't exactly be automated. 

Let's put these two functions in a loop that will check our changes in volume every 7 seconds and list the coins that are greater than our threshold over the specified interval. 

As a bonus, we will also add a clickable html link that will take you straight to the chart page of that asset on bittrex. For more fun, let's also add the ability to play a cash register sound if our tool discovers volume changes that meet our criteria. We can enable this sound by passing `alert_with_sound = True` into our function.


### Create The Function

In [ ]:
def monitor_volume_changes_and_alert_when_threshold_reached(live_exchange, volume_threshold, time_interval, alert_with_sound = False):
   
    while True:
        # you've seen these functions before
        volume_monitor_df = create_volume_monitor_dataframe(live_exchange, time_interval)
        coins_above_threshold = filter_volume_monitor_df_by_threshold(volume_monitor_df, volume_threshold)
        
        # check to see if there are any coins meeting our criteria
        if len(coins_above_threshold) > 0: 
            
            # play sound if alert_with_sound kw param is passed in
            if alert_with_sound: 
                
                # play cash register sound
                display(Audio("http://thecyberbuddy.com/sounds/CashRegister1.wav", autoplay = True)) 
            
            #  format the % change ouput
            coins_above_threshold = coins_above_threshold.applymap("{0:.2f}%".format) 
            
            # create link column with name of symbol in it
            coins_above_threshold['link'] = coins_above_threshold.index.values 
            
            # create a function to format our link so it is clickable
            def make_clickable(symbol):
                url = 'https://bittrex.com/Market/Index?MarketName='+symbol
                return '<a target="_blank" href="{}">{}</a>'.format(url, symbol)
            
            # clear jupyter's output - wait until we give it something new to display
            clear_output(wait = True) 
            # display dataframe with formatted link
            display(coins_above_threshold.style.format(make_clickable, subset=['link'])) 

        else:
            print('No coins found with pct_volume_change > %s%% in the last %s' % (volume_threshold, time_interval))
        
        # wait 7 seconds before checking the volumes again
        time.sleep(7)



### Run The Function
Now that we have created our master function that will alert us to changes in volume that satistfy our criteria. Lets implement it in a real-world scenario: finding coins that are being pumped!

Let's set our `time_interval` to `1T` and our `volume_threshold` to `1` (a change of one percent). When we run this function we will be alerted if a coin has seen a significant increase in volume in the last 20 seconds. You may want to specify `alert_with_sound = False` after running it a few times to silence the noise. If you do not see any coins, try lowering your volume threshold or increasing your time interval. 

Happy Hunting! :)

In [ ]:
monitor_volume_changes_and_alert_when_threshold_reached(
    live_bittrex, 
    volume_threshold = 1, 
    time_interval = '1T', 
    alert_with_sound = False
)

# Next Steps
- Try monitoring multiple time intervals at the same time and come up with an algorithm for alerting based on these changes:
    - e.g "alert when 10m interval is > 5% AND 1hr interval is >20%"
- Place orders instead of alerting
- Turn the display into a heatmap or better visual output
- Find correlations in the volume changes of different assets
- Conditionally style the output dataframe utilizing colors so you can visually see big/sudden changes
- Use jupyter/ipython widgets to create a GUI for using this tool (instead of typing in parameters)
- Be sure to check out https://github.com/LiveDataFrame/Tutorials for more tutorials utilizing live dataframe

_The above references an opinion and is for information purposes only.  It is not intended to be investment advice.  Seek a duly licensed professional for investment advice._